In [43]:
# Dependencies
import requests
from pprint import pprint
from config import api_key
import pandas as pd
import time
import json

url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

In [19]:
# Read JSON file to get company names and tickers

ticker_cat = pd.read_json("../DataScraping/ticker_categories.json")
ticker_cat.head()

,ticker,name,market,locale,primary_exchange,type,active,currency_name,cik,composite_figi,...,sic_description,ticker_root,homepage_url,total_employees,list_date,branding,share_class_shares_outstanding,weighted_shares_outstanding,round_lot,ticker_suffix
0,MMM,3M Company,stocks,us,XNYS,CS,True,usd,66740,BBG000BP52R2,...,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,MMM,https://www.3m.com,95000,1946-01-14,{'logo_url': 'https://api.polygon.io/v1/refere...,552740000,552742915,100,NaN
1,AOS,A.O. Smith Corporation,stocks,us,XNYS,CS,True,usd,91142,BBG000BC1L02,...,HOUSEHOLD APPLIANCES,AOS,https://www.aosmith.com,13700,1984-09-07,{'logo_url': 'https://api.polygon.io/v1/refere...,126870000,152775527,100,NaN
2,ABT,Abbott Laboratories,stocks,us,XNYS,CS,True,usd,1800,BBG000B9ZXB4,...,PHARMACEUTICAL PREPARATIONS,ABT,https://www.abbottinvestor.com,113000,1949-04-19,{'logo_url': 'https://api.polygon.io/v1/refere...,1743570000,1743573777,100,NaN
3,ABBV,ABBVIE INC.,stocks,us,XNYS,CS,True,usd,1551152,BBG0025Y4RY4,...,PHARMACEUTICAL PREPARATIONS,ABBV,http://www.abbvieinvestor.com,50000,2012-12-10,{'icon_url': 'https://api.polygon.io/v1/refere...,1768480000,1768480508,100,NaN
4,ABMD,Abiomed Inc,stocks,us,XNAS,CS,True,usd,815094,BBG000C101X4,...,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,ABMD,https://www.abiomed.com,2003,1987-07-30,{'logo_url': 'https://api.polygon.io/v1/refere...,45090000,45091184,100,NaN


In [26]:
# Cleaning (though, unnecessary)

ticker_fixed = ticker_cat[['ticker', 'name', 'locale', 'primary_exchange', 'sic_description', 'homepage_url']]
ticker_fixed.head()

,ticker,name,locale,primary_exchange,sic_description,homepage_url
0,MMM,3M Company,us,XNYS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.3m.com
1,AOS,A.O. Smith Corporation,us,XNYS,HOUSEHOLD APPLIANCES,https://www.aosmith.com
2,ABT,Abbott Laboratories,us,XNYS,PHARMACEUTICAL PREPARATIONS,https://www.abbottinvestor.com
3,ABBV,ABBVIE INC.,us,XNYS,PHARMACEUTICAL PREPARATIONS,http://www.abbvieinvestor.com
4,ABMD,Abiomed Inc,us,XNAS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.abiomed.com


In [27]:
ticker_renamed = ticker_fixed.rename(columns={"ticker":"Ticker", "name":"Name", "locale":"Locale", "primary_exchange":"Primary_Exchange", "sic_description":"Industry", "homepage_url":"Homepage"})
ticker_renamed.head()

,Ticker,Name,Locale,Primary_Exchange,Industry,Homepage
0,MMM,3M Company,us,XNYS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.3m.com
1,AOS,A.O. Smith Corporation,us,XNYS,HOUSEHOLD APPLIANCES,https://www.aosmith.com
2,ABT,Abbott Laboratories,us,XNYS,PHARMACEUTICAL PREPARATIONS,https://www.abbottinvestor.com
3,ABBV,ABBVIE INC.,us,XNYS,PHARMACEUTICAL PREPARATIONS,http://www.abbvieinvestor.com
4,ABMD,Abiomed Inc,us,XNAS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.abiomed.com


In [29]:
ticker_renamed['Headline'] = ""
ticker_renamed['Abstract'] = ""
ticker_renamed['Web_Url'] = ""
ticker_renamed['Date_Published'] = ""
ticker_renamed.head()

,Ticker,Name,Locale,Primary_Exchange,Industry,Homepage,Headline,Abstract,Web_Url,Date_Published
0,MMM,3M Company,us,XNYS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.3m.com,,,,
1,AOS,A.O. Smith Corporation,us,XNYS,HOUSEHOLD APPLIANCES,https://www.aosmith.com,,,,
2,ABT,Abbott Laboratories,us,XNYS,PHARMACEUTICAL PREPARATIONS,https://www.abbottinvestor.com,,,,
3,ABBV,ABBVIE INC.,us,XNYS,PHARMACEUTICAL PREPARATIONS,http://www.abbvieinvestor.com,,,,
4,ABMD,Abiomed Inc,us,XNAS,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,https://www.abiomed.com,,,,


In [73]:
# Get November 2022 articles for each company

articles_list = []

for index, row in ticker_renamed.iterrows():
    query = row['Name']
    begin_date = "20221101"
    end_date = "20221130"
    
    query_url = url  + "begin_date=" + begin_date + "&end_date=" + end_date + "&fl=docs&fq=news_desk:('Financial')&q=" + query + "&sort=newest&api-key=" + api_key
    articles = requests.get(query_url).json()
    time.sleep(6)
    for article in articles:
        articles_list.append(article['response']['docs'])

In [56]:
with open ('NYT_fixed.json', 'w') as f:
    json.dump(articles_list, f)

In [138]:
# Run the other months (change month parameters for each run) - Each run takes 54 minutes

articles_list = []

for index, row in ticker_renamed.iterrows():
    query = row['Name']
    begin_date = "20220601"
    end_date = "20220630"
    
    query_url = url  + "begin_date=" + begin_date + "&end_date=" + end_date + "&q=" + query + "&sort=newest&api-key=" + api_key
    articles = requests.get(query_url).json()
    time.sleep(6)
    for article in articles:
        articles_list.append(articles['response']['docs'])

In [131]:
with open ('NYT_Oct.json', 'w') as f:
    json.dump(articles_list, f)

In [133]:
with open ('NYT_Nov.json', 'w') as f:
    json.dump(articles_list, f)

In [135]:
with open ('NYT_Aug.json', 'w') as f:
    json.dump(articles_list, f)

In [137]:
with open ('NYT_July.json', 'w') as f:
    json.dump(articles_list, f)

In [139]:
with open ('NYT_June.json', 'w') as f:
    json.dump(articles_list, f)